In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pymc3 import Model, Normal, HalfNormal,Lognormal
from pymc3 import NUTS, sample
from scipy import optimize
from pymc3 import traceplot
from pymc3 import summary
import theano.tensor as tt

df = pd.read_csv("data/data-conc-2.txt")
df = df.drop("PersonID", axis=1)
Y_nt = df.values
n_patient = Y_nt.shape[0]
times = np.reshape(np.array([1,2,4,8,12,24]),(1,-1))

basic_model = Model()

with basic_model:
    #全体平均
    a_0 = Normal('a_0', mu=0, sd=10)
    b_0 = Normal('b_0', mu=0, sd=10)
    #全体分散
    s_ag = HalfNormal('sigma_a', sd=10)
    s_bg = HalfNormal('sigma_b', sd=10)

    #個人パラメータ
    a = Lognormal('a', mu=a_0,sd=s_ag , shape=[n_patient,1])
    b = Lognormal('b', mu=b_0, sd=s_bg , shape=[n_patient,1])
    
    #個人分散
    s_Y = HalfNormal('sigma_Y', sd=10)
    
    #likelihood 
    mu = 1-tt.exp( -tt.dot(b,times) )
    mu = a*mu
    Y_obs = Normal('Y_obs', mu=mu, sd=s_Y, observed=Y_nt)
    
    #サンプリング
    trace = sample(100)
    summary(trace)

Auto-assigning NUTS sampler...
Initializing NUTS using advi...
Average ELBO = -338.24: 100%|██████████| 200000/200000 [00:59<00:00, 3381.96it/s]
Finished [100%]: Average ELBO = -337.6
100%|██████████| 100/100 [00:22<00:00,  4.38it/s]


a_0:

  Mean             SD               MC Error         95% HPD interval
  -------------------------------------------------------------------
  
  2.874            0.143            0.014            [2.489, 3.087]

  Posterior quantiles:
  2.5            25             50             75             97.5
  |--------------|==============|==============|--------------|
  
  2.516          2.817          2.901          2.955          3.169


b_0:

  Mean             SD               MC Error         95% HPD interval
  -------------------------------------------------------------------
  
  -0.952           0.930            0.093            [-1.670, 2.542]

  Posterior quantiles:
  2.5            25             50             75             97.5
  |--------------|==============|==============|--------------|
  
  -1.670         -1.280         -1.179         -1.047         2.675


sigma_a:

  Mean             SD               MC Error         95% HPD interval
  --------------------------